In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode
import csv
import matplotlib.dates
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from datetime import *
import sys, urllib, json
import urllib.request
from sklearn.preprocessing import *

%matplotlib inline

In [2]:
# 导入数据
df = pd.read_csv('fusai_train_20180227.txt', sep='\t')

print(df.info())
print(df.describe())
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8971 entries, 0 to 8970
Data columns (total 4 columns):
date           8971 non-null int64
day_of_week    8971 non-null int64
brand          8971 non-null int64
cnt            8971 non-null int64
dtypes: int64(4)
memory usage: 280.4 KB
None
              date  day_of_week        brand          cnt
count  8971.000000  8971.000000  8971.000000  8971.000000
mean    581.635938     3.484896     5.527143   314.299075
std     318.325448     1.732524     2.953752   255.770966
min       1.000000     1.000000     1.000000     4.000000
25%     310.000000     2.000000     3.000000   146.000000
50%     600.000000     3.000000     6.000000   265.000000
75%     858.000000     5.000000     8.000000   416.000000
max    1107.000000     7.000000    10.000000  3164.000000
   date  day_of_week  brand  cnt
0     1            2      1   31
1     1            2      6    6
2     1            2      9   15
3     2            3      4   20
4     2            3  

In [3]:
# 标记周数
def tickWeek(df, start):
    preVal = df[:-1]['day_of_week'].values
    preVal = np.insert(preVal, 0, preVal[0]-1)
    df['new_week'] = (preVal >= df.day_of_week)        #判断是否是新的一周
    weekList = list()
    week = start
    for item in df['new_week']:
        week += (1 if item else 0)
        weekList.append(week)
    df['week'] = weekList
    return df

In [4]:
# 标记周数+修正日期
dataDf = df.drop_duplicates(['date', 'day_of_week'])
dataDf = tickWeek(dataDf, 0)
dataDf['day'] = dataDf['week']*7 + dataDf['day_of_week']
print(dataDf.head(10))

    date  day_of_week  brand  cnt  new_week  week  day
0      1            2      1   31     False     0    2
3      2            3      4   20     False     0    3
7      3            4      4   16     False     0    4
10     4            5      1  898     False     0    5
19     5            6      1  842     False     0    6
28     6            7      1  649     False     0    7
37     7            1      1  574      True     1    8
46     8            2      1  842     False     1    9
55     9            3      1  640     False     1   10
64    10            4      1  389     False     1   11


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [5]:
# 给数据添加日期字段
def addGuessDate(df, startYear, startMonth, startDay):
    startDate = date(startYear, startMonth, startDay)
    df['guess_date'] = df['day'].map(lambda x: startDate + timedelta(days=x))
    df['guess_date'] = pd.to_datetime(df['guess_date'])
    return df

# 尝试给数据加日期
dataDf = addGuessDate(dataDf,2012,12,30)
dataDf = dataDf.drop('new_week', 1)

print(dataDf.head(20))

     date  day_of_week  brand  cnt  week  day guess_date
0       1            2      1   31     0    2 2013-01-01
3       2            3      4   20     0    3 2013-01-02
7       3            4      4   16     0    4 2013-01-03
10      4            5      1  898     0    5 2013-01-04
19      5            6      1  842     0    6 2013-01-05
28      6            7      1  649     0    7 2013-01-06
37      7            1      1  574     1    8 2013-01-07
46      8            2      1  842     1    9 2013-01-08
55      9            3      1  640     1   10 2013-01-09
64     10            4      1  389     1   11 2013-01-10
73     11            5      1  274     1   12 2013-01-11
82     12            6      1   94     1   13 2013-01-12
89     13            7      1   27     1   14 2013-01-13
92     14            1      1  449     2   15 2013-01-14
101    15            2      1  811     2   16 2013-01-15
110    16            3      1  611     2   17 2013-01-16
118    17            4      1  

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [12]:
df['guess_date'] = df['date'].map(lambda x: dataDf['guess_date'][dataDf.date == x].values[0])
df['week'] = df['date'].map(lambda x: dataDf['week'][dataDf.date == x].values[0])
print(df.head(10))

   date  day_of_week  brand  cnt guess_date  week
0     1            2      1   31 2013-01-01     0
1     1            2      6    6 2013-01-01     0
2     1            2      9   15 2013-01-01     0
3     2            3      4   20 2013-01-02     0
4     2            3      6    6 2013-01-02     0
5     2            3      7   30 2013-01-02     0
6     2            3     10   48 2013-01-02     0
7     3            4      4   16 2013-01-03     0
8     3            4      6    4 2013-01-03     0
9     3            4      8   23 2013-01-03     0


In [13]:
#处理测试集A相关数据
df_test_A = pd.read_csv('fusai_test_A_20180227.txt', sep='\t')

df_test = df_test_A.drop_duplicates(['date', 'day_of_week'])
df_test = tickWeek(df_test, dataDf.loc[dataDf.index[-1], 'week'])
df_test['day'] = df_test['week'] * 7 + df_test['day_of_week']

df_test = addGuessDate(df_test,2012,12,30)
df_test = df_test.drop('new_week', 1)
df_test_A['week'] = df_test_A['date'].map(lambda x: df_test['week'][df_test.date == x].values[0])
df_test_A['guess_date'] = df_test_A['date'].map(lambda x: df_test['guess_date'][df_test.date == x].values[0])

answer_predict = pd.read_csv('fusai_answer_a_20180307.txt', sep='\t', header=None)
df_test_A['cnt'] = answer_predict[:][2].values

print(df_test_A.head(10))

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

   date  day_of_week  brand  week guess_date  cnt
0  1107            4      7   174 2016-05-05  232
1  1107            4      8   174 2016-05-05  583
2  1107            4      9   174 2016-05-05  466
3  1107            4     10   174 2016-05-05  288
4  1108            5      1   174 2016-05-06  490
5  1108            5      2   174 2016-05-06  318
6  1108            5      3   174 2016-05-06  285
7  1108            5      4   174 2016-05-06  380
8  1108            5      5   174 2016-05-06  470
9  1108            5      6   174 2016-05-06  378


In [20]:
#将训练集和测试集A的数据组合起来作为训练集
train_df_B = pd.concat([df, df_test_A])
print(len(train_df_B))
train_df_B = train_df_B.drop_duplicates(['brand', 'cnt', 'date'])
print(len(train_df_B))
print(train_df_B.head(10))

10260
10259
   brand  cnt  date  day_of_week guess_date  week
0      1   31     1            2 2013-01-01     0
1      6    6     1            2 2013-01-01     0
2      9   15     1            2 2013-01-01     0
3      4   20     2            3 2013-01-02     0
4      6    6     2            3 2013-01-02     0
5      7   30     2            3 2013-01-02     0
6     10   48     2            3 2013-01-02     0
7      4   16     3            4 2013-01-03     0
8      6    4     3            4 2013-01-03     0
9      8   23     3            4 2013-01-03     0


In [24]:
# 尝试得到缺失的日期数据
B_fillingDf = dataDf[:0][['date', 'brand', 'cnt', 'day_of_week', 'week', 'guess_date']]
print(B_fillingDf.head(10))

Empty DataFrame
Columns: [date, brand, cnt, day_of_week, week, guess_date]
Index: []


In [25]:
#定义补充缺失日期的函数
def fillData(fillingDf, df, brand, startYear, startMonth, startDay):
    preDate = df[:-1]['guess_date'].values
    preDate = np.insert(preDate, 0, datetime(startYear, startMonth, startDay))
    df['pre_date'] = preDate
    df['day_diff'] = (df['guess_date'] - df['pre_date']).map(lambda x : x.days)
    # 获取缺失日期的dataFrame
    for i in range(len(df)):
        cnt = df.iloc[[i], [1]].values
        day_of_week = df.iloc[[i], [3]].values
        week = df.iloc[[i], [5]].values
        guess_date = df.iloc[[i], [4]].values
        pre_date = df.iloc[[i], [6]].values
        delta_day = df.iloc[[i], [7]].values[0][0]
        date = df.iloc[[i], [2]].values[0][0]
        if delta_day > 1:
            for n in range(delta_day - 1):
                fill_date = (guess_date - (n + 1) * 24 * 60 * 60 * 1000000000)[0][0]
                fill_day_of_week = (day_of_week - (n + 1))[0][0]
                fill_week = week[0][0]
                if fill_day_of_week < 1:
                    fill_week = (int(abs(fill_day_of_week) / 7) - 1 + week)[0][0]
                    fill_day_of_week = (fill_day_of_week % 7)
                    if fill_day_of_week == 0:
                        fill_day_of_week = 7
                fill_cnt = 0
                fill_day = date - n - 1
                fill_brand = brand
                fill_row = pd.DataFrame([[fill_day, fill_brand, fill_cnt, fill_day_of_week, fill_week, fill_date]], columns=['date', 'brand', 'cnt', 'day_of_week', 'week', 'guess_date'])
                fillingDf = fillingDf.append(fill_row)
    return fillingDf

B_fillingDf = fillData(B_fillingDf, train_df_B[:][train_df_B.brand == 1], 1, 2013, 1, 1)
B_fillingDf = fillData(B_fillingDf, train_df_B[:][train_df_B.brand == 2], 2, 2013, 1, 1)
B_fillingDf = fillData(B_fillingDf, train_df_B[:][train_df_B.brand == 3], 3, 2013, 1, 1)
B_fillingDf = fillData(B_fillingDf, train_df_B[:][train_df_B.brand == 4], 4, 2013, 1, 1)
B_fillingDf = fillData(B_fillingDf, train_df_B[:][train_df_B.brand == 6], 6, 2013, 1, 1)
B_fillingDf = fillData(B_fillingDf, train_df_B[:][train_df_B.brand == 7], 7, 2013, 1, 1)
B_fillingDf = fillData(B_fillingDf, train_df_B[:][train_df_B.brand == 8], 8, 2013, 1, 1)
B_fillingDf = fillData(B_fillingDf, train_df_B[:][train_df_B.brand == 9], 9, 2013, 1, 1)
B_fillingDf = fillData(B_fillingDf, train_df_B[:][train_df_B.brand == 10], 10, 2013, 1, 1)
B_fillingDf = fillData(B_fillingDf, train_df_B[:][train_df_B.brand == 5], 5, 2014, 7, 4)

print(B_fillingDf.head(10))

   date  brand  cnt  day_of_week  week guess_date
0     3      1    0            4     0 2013-01-03
0     2      1    0            3     0 2013-01-02
0    34      1    0            7     4 2013-02-03
0    41      1    0            5     6 2013-02-15
0    40      1    0            4     6 2013-02-14
0    39      1    0            3     6 2013-02-13
0    38      1    0            2     6 2013-02-12
0    37      1    0            1     6 2013-02-11
0    36      1    0            7     5 2013-02-10
0    35      1    0            6     5 2013-02-09


In [26]:
# 尝试将原本训练集和缺失日期训练集合并
full_train_df = pd.concat([train_df_B[:][['date', 'brand', 'cnt', 'day_of_week', 'week', 'guess_date']], B_fillingDf[:][['date', 'brand', 'cnt', 'day_of_week', 'week', 'guess_date']]])
#将数据集按日期升序重新排序
full_train_df = full_train_df.sort_values(by='date', ascending=True)
print(full_train_df.head(10))
print(len(full_train_df))

   date  brand  cnt  day_of_week  week guess_date
0     1      1   31            2     0 2013-01-01
1     1      6    6            2     0 2013-01-01
2     1      9   15            2     0 2013-01-01
0     2      9    0            3     0 2013-01-02
0     2      8    0            3     0 2013-01-02
0     2      3    0            3     0 2013-01-02
0     2      2    0            3     0 2013-01-02
5     2      7   30            3     0 2013-01-02
4     2      6    6            3     0 2013-01-02
3     2      4   20            3     0 2013-01-02
13245


In [27]:
#给数据添加年份字段
def getDateYear(df, colName):
    df['date_year'] = df[colName].map(lambda x: x.year)
    return df

#给数据添加月份字段
def getDateMonth(df, colName):
    df['date_month'] = df[colName].map(lambda x: x.month)
    return df

full_train_df = getDateYear(full_train_df, 'guess_date')
full_train_df = getDateMonth(full_train_df, 'guess_date')
print(full_train_df.head(10))

   date  brand  cnt  day_of_week  week guess_date  date_year  date_month
0     1      1   31            2     0 2013-01-01       2013           1
1     1      6    6            2     0 2013-01-01       2013           1
2     1      9   15            2     0 2013-01-01       2013           1
0     2      9    0            3     0 2013-01-02       2013           1
0     2      8    0            3     0 2013-01-02       2013           1
0     2      3    0            3     0 2013-01-02       2013           1
0     2      2    0            3     0 2013-01-02       2013           1
5     2      7   30            3     0 2013-01-02       2013           1
4     2      6    6            3     0 2013-01-02       2013           1
3     2      4   20            3     0 2013-01-02       2013           1


In [28]:
# 尝试利用holiday文件获取日期是否节假日特征
def get_day_type(df, colName):
    holiday_df = pd.read_csv('holiday.csv')
    df['date_property'] = 0
    df['guess_date_str'] = df[colName].map(lambda x: x.strftime('%Y-%m-%d'))
    for guess_date_str in df['guess_date_str']:
        df.date_property[df.guess_date_str == guess_date_str] = holiday_df['holiday_type'][holiday_df.date == guess_date_str].values
    return df

full_train_df = get_day_type(full_train_df, 'guess_date')
print(full_train_df.head(10))

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


   date  brand  cnt  day_of_week  week guess_date  date_year  date_month  \
0     1      1   31            2     0 2013-01-01       2013           1   
1     1      6    6            2     0 2013-01-01       2013           1   
2     1      9   15            2     0 2013-01-01       2013           1   
0     2      9    0            3     0 2013-01-02       2013           1   
0     2      8    0            3     0 2013-01-02       2013           1   
0     2      3    0            3     0 2013-01-02       2013           1   
0     2      2    0            3     0 2013-01-02       2013           1   
5     2      7   30            3     0 2013-01-02       2013           1   
4     2      6    6            3     0 2013-01-02       2013           1   
3     2      4   20            3     0 2013-01-02       2013           1   

   date_property guess_date_str  
0              2     2013-01-01  
1              2     2013-01-01  
2              2     2013-01-01  
0              2     2013-0

In [29]:
#尝试使用另一个比赛的销售量数据作为特征
def getSaleQuantity(sale_quantity_data, date):
    sale_date_this_month = date.year * 100 + date.month
    if date.month == 1:
        sale_date_last_month = (date.year - 1) * 100 + 12
    else:
        sale_date_last_month = sale_date_this_month - 1
    sale_quantity_this_month = sale_quantity_data.sale_quantity[sale_quantity_data.sale_date == sale_date_this_month].sum()
    sale_quantity_last_month = sale_quantity_data.sale_quantity[sale_quantity_data.sale_date == sale_date_last_month].sum()
    result = sale_quantity_last_month
    return result

#对数据集填充销售量数据
def fillSaleQuantity(df, sale_quantity_data):
    df['sale_quantity'] = 0
    for date in df['guess_date']:
        df.sale_quantity[(df.date_year == date.year) & (df.date_month == date.month)] = getSaleQuantity(sale_quantity_data, date)
    return df

sale_quantity_data = pd.read_csv('yancheng_train_20171226.csv')

#给训练集添加sale_quantity字段
full_train_df = fillSaleQuantity(full_train_df, sale_quantity_data)
print(full_train_df.head(10))

/usr/local/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


   date  brand  cnt  day_of_week  week guess_date  date_year  date_month  \
0     1      1   31            2     0 2013-01-01       2013           1   
1     1      6    6            2     0 2013-01-01       2013           1   
2     1      9   15            2     0 2013-01-01       2013           1   
0     2      9    0            3     0 2013-01-02       2013           1   
0     2      8    0            3     0 2013-01-02       2013           1   
0     2      3    0            3     0 2013-01-02       2013           1   
0     2      2    0            3     0 2013-01-02       2013           1   
5     2      7   30            3     0 2013-01-02       2013           1   
4     2      6    6            3     0 2013-01-02       2013           1   
3     2      4   20            3     0 2013-01-02       2013           1   

   date_property guess_date_str  sale_quantity  
0              2     2013-01-01          20746  
1              2     2013-01-01          20746  
2              2

In [30]:
#给数据添加属于当月上旬还是下旬特征数据
def addDividedMonth(df, colName):
    df[colName] = df.guess_date.map(lambda x: 'early' if (x.day <= 15) else 'late')
    return df

full_train_df = addDividedMonth(full_train_df, 'dividedMonth')
print(full_train_df.head(10))

   date  brand  cnt  day_of_week  week guess_date  date_year  date_month  \
0     1      1   31            2     0 2013-01-01       2013           1   
1     1      6    6            2     0 2013-01-01       2013           1   
2     1      9   15            2     0 2013-01-01       2013           1   
0     2      9    0            3     0 2013-01-02       2013           1   
0     2      8    0            3     0 2013-01-02       2013           1   
0     2      3    0            3     0 2013-01-02       2013           1   
0     2      2    0            3     0 2013-01-02       2013           1   
5     2      7   30            3     0 2013-01-02       2013           1   
4     2      6    6            3     0 2013-01-02       2013           1   
3     2      4   20            3     0 2013-01-02       2013           1   

   date_property guess_date_str  sale_quantity dividedMonth  
0              2     2013-01-01          20746        early  
1              2     2013-01-01        

In [31]:
#给数据集添加是否是休息日或节假日后一天特征
def addRestdayAfterOneDay(df, colName, insert):
    preVal = df[:-1]['date_property'].values
    preVal = np.insert(preVal, 0, insert)
    df['pre_date_property'] = preVal
    df[colName] = 0
    df[colName][(df.date_property == 0) & ((df.pre_date_property == 2) | (df.pre_date_property == 1))] = 1
    return df

dataDf = full_train_df.drop_duplicates(['guess_date'])
dataDf = get_day_type(dataDf, 'guess_date')
dataDf = addRestdayAfterOneDay(dataDf, 'after_restday_one', 0)

full_train_df['after_restday_one'] = full_train_df['date'].map(lambda x: dataDf['after_restday_one'][dataDf.date == x].values[0])

print(full_train_df.head(10))

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#in

   date  brand  cnt  day_of_week  week guess_date  date_year  date_month  \
0     1      1   31            2     0 2013-01-01       2013           1   
1     1      6    6            2     0 2013-01-01       2013           1   
2     1      9   15            2     0 2013-01-01       2013           1   
0     2      9    0            3     0 2013-01-02       2013           1   
0     2      8    0            3     0 2013-01-02       2013           1   
0     2      3    0            3     0 2013-01-02       2013           1   
0     2      2    0            3     0 2013-01-02       2013           1   
5     2      7   30            3     0 2013-01-02       2013           1   
4     2      6    6            3     0 2013-01-02       2013           1   
3     2      4   20            3     0 2013-01-02       2013           1   

   date_property guess_date_str  sale_quantity dividedMonth  after_restday_one  
0              2     2013-01-01          20746        early                  0  
1

In [32]:
#给数据集添加是否是节假日后一天特征
def addHolidayAfterOneDay(df, colName, index):
    df[colName] = 0
    length = len(df[colName])
    for i in range(length):
        if df.loc[(i + index)]['date_property'] == 2:
            for j in range(30):
                if (i + j) < length:
                    if df.loc[(i + j + index)]['date_property'] == 0:
                        df[colName].loc[(i + j + index)] = 1
                        break
                else:
                    break
        else:
            continue
    return df

#给数据集手动添加是否是节假日后一天特征
def addHolidayAfterOneDayByDate(df, colName, date_set):
    date_set = pd.to_datetime(date_set)
    df[colName] = 0
    for date in date_set:
        df[colName][df.guess_date == date] = 1
    return df

# dataDf = addHolidayAfterOneDay(dataDf, 'after_holiday_one', 1)
train_date_set = ['2013-01-04', '2013-02-16', '2013-04-07', '2013-05-02', '2013-06-13', '2013-09-22', '2013-10-08',
                 '2014-01-02', '2014-02-07', '2014-04-08', '2014-05-04', '2014-06-03', '2014-09-09', '2014-10-08',
                 '2015-01-04', '2015-02-25', '2015-04-07', '2015-05-04', '2015-06-23', '2015-09-06', '2015-09-28', '2015-10-08',
                 '2016-01-04', '2016-02-14', '2016-04-05', '2016-05-03', '2016-06-12', '2016-09-18', '2016-10-08']
full_train_df = addHolidayAfterOneDayByDate(full_train_df, 'after_holiday_one', train_date_set)

print(full_train_df.head(20))

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


   date  brand  cnt  day_of_week  week guess_date  date_year  date_month  \
0     1      1   31            2     0 2013-01-01       2013           1   
1     1      6    6            2     0 2013-01-01       2013           1   
2     1      9   15            2     0 2013-01-01       2013           1   
0     2      9    0            3     0 2013-01-02       2013           1   
0     2      8    0            3     0 2013-01-02       2013           1   
0     2      3    0            3     0 2013-01-02       2013           1   
0     2      2    0            3     0 2013-01-02       2013           1   
5     2      7   30            3     0 2013-01-02       2013           1   
4     2      6    6            3     0 2013-01-02       2013           1   
3     2      4   20            3     0 2013-01-02       2013           1   
6     2     10   48            3     0 2013-01-02       2013           1   
0     2      1    0            3     0 2013-01-02       2013           1   
0     3     

In [33]:
#给数据集添加是否是节假轮休日特征，即day_of_week等于1-5，date_property等于1或者2的日期
def addIsHolidayRestDay(df, colName):
    df[colName] = 0
    df[colName][((df.date_property == 1) | (df.date_property == 2)) & (df.day_of_week <= 5)] = 1
    return df

full_train_df = addIsHolidayRestDay(full_train_df, 'is_holi_restday')
print(full_train_df.head(10))

   date  brand  cnt  day_of_week  week guess_date  date_year  date_month  \
0     1      1   31            2     0 2013-01-01       2013           1   
1     1      6    6            2     0 2013-01-01       2013           1   
2     1      9   15            2     0 2013-01-01       2013           1   
0     2      9    0            3     0 2013-01-02       2013           1   
0     2      8    0            3     0 2013-01-02       2013           1   
0     2      3    0            3     0 2013-01-02       2013           1   
0     2      2    0            3     0 2013-01-02       2013           1   
5     2      7   30            3     0 2013-01-02       2013           1   
4     2      6    6            3     0 2013-01-02       2013           1   
3     2      4   20            3     0 2013-01-02       2013           1   

   date_property guess_date_str  sale_quantity dividedMonth  \
0              2     2013-01-01          20746        early   
1              2     2013-01-01      

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [34]:
#给数据集手动添加是否是元旦假期特征
def addIsNewYearDay(df, colName, date_set):
    df[colName] = 0
    date_set = pd.to_datetime(date_set)
    for date in date_set:
        df[colName][df.guess_date == date] = 1
    return df

train_newYearDay_set = ['2013-01-01', '2013-01-02', '2013-01-03', '2014-01-01', '2015-01-01', '2015-01-02', '2015-01-03', 
                        '2016-01-01', '2016-01-02', '2016-01-03']
full_train_df = addIsNewYearDay(full_train_df, 'is_newYearDay', train_newYearDay_set)

print(full_train_df.head(10))

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


   date  brand  cnt  day_of_week  week guess_date  date_year  date_month  \
0     1      1   31            2     0 2013-01-01       2013           1   
1     1      6    6            2     0 2013-01-01       2013           1   
2     1      9   15            2     0 2013-01-01       2013           1   
0     2      9    0            3     0 2013-01-02       2013           1   
0     2      8    0            3     0 2013-01-02       2013           1   
0     2      3    0            3     0 2013-01-02       2013           1   
0     2      2    0            3     0 2013-01-02       2013           1   
5     2      7   30            3     0 2013-01-02       2013           1   
4     2      6    6            3     0 2013-01-02       2013           1   
3     2      4   20            3     0 2013-01-02       2013           1   

   date_property guess_date_str  sale_quantity dividedMonth  \
0              2     2013-01-01          20746        early   
1              2     2013-01-01      

In [35]:
#给数据集手动添加是否春节假期特征
def addIsChineseNewYearDay(df, colName, date_set):
    df[colName] = 0
    date_set = pd.to_datetime(date_set)
    for date in date_set:
        df[colName][df.guess_date == date] = 1
    return df

train_ChineseNewYearDay_set = ['2013-02-09', '2013-02-10', '2013-02-11', '2013-02-12', '2013-02-13', '2013-02-14', '2013-02-15', 
                        '2014-01-31', '2014-02-01', '2014-02-02', '2014-02-03', '2014-02-04', '2014-02-05', '2014-02-06', 
                        '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-21', '2015-02-22', '2015-02-23', '2015-02-24',
                        '2016-02-07', '2016-02-08', '2016-02-09', '2016-02-10', '2016-02-11', '2016-02-12', '2016-02-13']
full_train_df = addIsChineseNewYearDay(full_train_df, 'is_ChineseNewYearDay', train_ChineseNewYearDay_set)

print(full_train_df.head(10))

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


   date  brand  cnt  day_of_week  week guess_date  date_year  date_month  \
0     1      1   31            2     0 2013-01-01       2013           1   
1     1      6    6            2     0 2013-01-01       2013           1   
2     1      9   15            2     0 2013-01-01       2013           1   
0     2      9    0            3     0 2013-01-02       2013           1   
0     2      8    0            3     0 2013-01-02       2013           1   
0     2      3    0            3     0 2013-01-02       2013           1   
0     2      2    0            3     0 2013-01-02       2013           1   
5     2      7   30            3     0 2013-01-02       2013           1   
4     2      6    6            3     0 2013-01-02       2013           1   
3     2      4   20            3     0 2013-01-02       2013           1   

   date_property guess_date_str  sale_quantity dividedMonth  \
0              2     2013-01-01          20746        early   
1              2     2013-01-01      

In [36]:
#给数据集手动添加是否国庆假期特征
def addIsNationalDay(df, colName, date_set):
    df[colName] = 0
    date_set = pd.to_datetime(date_set)
    for date in date_set:
        df[colName][df.guess_date == date] = 1
    return df

train_NationalDay_set = ['2013-10-01', '2013-10-02', '2013-10-03', '2013-10-04', '2013-10-05', '2013-10-06', '2013-10-07', 
                        '2014-10-01', '2014-10-02', '2014-10-03', '2014-10-04', '2014-10-05', '2014-10-06', '2014-10-07', 
                        '2015-10-01', '2015-10-02', '2015-10-03', '2015-10-04', '2015-10-05', '2015-10-06', '2015-10-07',
                        '2016-10-01', '2016-10-02', '2016-10-03', '2016-10-04', '2016-10-05', '2016-10-06', '2016-10-07']
full_train_df = addIsNationalDay(full_train_df, 'is_NationalDay', train_NationalDay_set)

print(full_train_df.head(10))

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


   date  brand  cnt  day_of_week  week guess_date  date_year  date_month  \
0     1      1   31            2     0 2013-01-01       2013           1   
1     1      6    6            2     0 2013-01-01       2013           1   
2     1      9   15            2     0 2013-01-01       2013           1   
0     2      9    0            3     0 2013-01-02       2013           1   
0     2      8    0            3     0 2013-01-02       2013           1   
0     2      3    0            3     0 2013-01-02       2013           1   
0     2      2    0            3     0 2013-01-02       2013           1   
5     2      7   30            3     0 2013-01-02       2013           1   
4     2      6    6            3     0 2013-01-02       2013           1   
3     2      4   20            3     0 2013-01-02       2013           1   

   date_property guess_date_str  sale_quantity dividedMonth  \
0              2     2013-01-01          20746        early   
1              2     2013-01-01      

In [37]:
# 给数据集添加当天上牌车辆品牌个数特征
def addPredictBrandsNumber(df, colName):
    df[colName] = df['date'].map(lambda x: len(df['date'][df.date == x]))
    return df

full_train_df = addPredictBrandsNumber(full_train_df, 'brands_number')
print(full_train_df.head(10))

   date  brand  cnt  day_of_week  week guess_date  date_year  date_month  \
0     1      1   31            2     0 2013-01-01       2013           1   
1     1      6    6            2     0 2013-01-01       2013           1   
2     1      9   15            2     0 2013-01-01       2013           1   
0     2      9    0            3     0 2013-01-02       2013           1   
0     2      8    0            3     0 2013-01-02       2013           1   
0     2      3    0            3     0 2013-01-02       2013           1   
0     2      2    0            3     0 2013-01-02       2013           1   
5     2      7   30            3     0 2013-01-02       2013           1   
4     2      6    6            3     0 2013-01-02       2013           1   
3     2      4   20            3     0 2013-01-02       2013           1   

   date_property guess_date_str  sale_quantity dividedMonth  \
0              2     2013-01-01          20746        early   
1              2     2013-01-01      

In [38]:
#给数据集添加是否是节假周末的特征
def addIsHoliWeekend(df, colName, after_date_property_insert, pre_isHoliWeekend_insert, after_isHoliWeekend_insert):
    df[colName] = 0
    df[colName][(df.day_of_week == 6) & ((df.date_property == 2) | ((df.date_property == 1) & (df.pre_date_property == 2)))] = 1
    afterVal = df['date_property'].values
    afterVal = afterVal[1:]
    afterVal = np.insert(afterVal, -1, after_date_property_insert)
    df['after_date_property'] = afterVal
    df[colName][(df.day_of_week == 7) & ((df.date_property == 2) | ((df.date_property == 1) & (df.after_date_property == 2)))] = 1
    
    pre_isHolidayWeekend = df[colName].values
    pre_isHolidayWeekend = pre_isHolidayWeekend[:(- 1)]
    pre_isHolidayWeekend = np.insert(pre_isHolidayWeekend, 0, pre_isHoliWeekend_insert)
    df['pre_isHolidayWeekend'] = pre_isHolidayWeekend
    
    after_isHolidayWeekend = df[colName].values
    after_isHolidayWeekend = after_isHolidayWeekend[1:]
    after_isHolidayWeekend = np.insert(after_isHolidayWeekend, -1, after_isHoliWeekend_insert)
    df['after_isHolidayWeekend'] = after_isHolidayWeekend
    
    df[colName][(df.day_of_week == 6) & (df.after_isHolidayWeekend == 1) & (df.date_property == 1)] = 1
    df[colName][(df.day_of_week == 7) & (df.pre_isHolidayWeekend == 1) & (df.date_property == 1)] = 1
    return df

dataDf = addIsHoliWeekend(dataDf, 'isHolidayWeekend', 0, 0, 0)

full_train_df['isHolidayWeekend'] = full_train_df['date'].map(lambda x: dataDf['isHolidayWeekend'][dataDf.date == x].values[0])

print(full_train_df.head(10))

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

   date  brand  cnt  day_of_week  week guess_date  date_year  date_month  \
0     1      1   31            2     0 2013-01-01       2013           1   
1     1      6    6            2     0 2013-01-01       2013           1   
2     1      9   15            2     0 2013-01-01       2013           1   
0     2      9    0            3     0 2013-01-02       2013           1   
0     2      8    0            3     0 2013-01-02       2013           1   
0     2      3    0            3     0 2013-01-02       2013           1   
0     2      2    0            3     0 2013-01-02       2013           1   
5     2      7   30            3     0 2013-01-02       2013           1   
4     2      6    6            3     0 2013-01-02       2013           1   
3     2      4   20            3     0 2013-01-02       2013           1   

   date_property guess_date_str  sale_quantity dividedMonth  \
0              2     2013-01-01          20746        early   
1              2     2013-01-01      

In [39]:
#给数据集添加是否是纯休周末特征
def addIsPureWeekend(df, colName):
    df[colName] = 0
    df[colName][(df.date_property == 1) & (df.isHolidayWeekend == 0) & ((df.day_of_week == 6) | (df.day_of_week == 7))] = 1
    return df

full_train_df = addIsPureWeekend(full_train_df, 'isPureWeekend')
print(full_train_df.head(10))

   date  brand  cnt  day_of_week  week guess_date  date_year  date_month  \
0     1      1   31            2     0 2013-01-01       2013           1   
1     1      6    6            2     0 2013-01-01       2013           1   
2     1      9   15            2     0 2013-01-01       2013           1   
0     2      9    0            3     0 2013-01-02       2013           1   
0     2      8    0            3     0 2013-01-02       2013           1   
0     2      3    0            3     0 2013-01-02       2013           1   
0     2      2    0            3     0 2013-01-02       2013           1   
5     2      7   30            3     0 2013-01-02       2013           1   
4     2      6    6            3     0 2013-01-02       2013           1   
3     2      4   20            3     0 2013-01-02       2013           1   

   date_property guess_date_str      ...        dividedMonth  \
0              2     2013-01-01      ...               early   
1              2     2013-01-01    

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [40]:
# 导出训练集预处理结果
def exportResult(df, fileName):
    df.to_csv('./%s.csv' % fileName, header=True, index=False)

In [41]:
#处理测试集B相关数据
df_test_B = pd.read_csv('fusai_test_B_20180227.txt', sep='\t')

df_test = df_test_B.drop_duplicates(['date', 'day_of_week'])
df_test = tickWeek(df_test, dataDf.loc[dataDf.index[-1], 'week'])
df_test['day'] = df_test['week'] * 7 + df_test['day_of_week']

df_test = addGuessDate(df_test,2012,12,30)
df_test = df_test.drop('new_week', 1)
df_test_B['week'] = df_test_B['date'].map(lambda x: df_test['week'][df_test.date == x].values[0])
df_test_B['guess_date'] = df_test_B['date'].map(lambda x: df_test['guess_date'][df_test.date == x].values[0])

df_test_B = getDateYear(df_test_B, 'guess_date')
df_test_B = getDateMonth(df_test_B, 'guess_date')

df_test_B = get_day_type(df_test_B, 'guess_date')

df_test_B = fillSaleQuantity(df_test_B, sale_quantity_data)

df_test_B = addDividedMonth(df_test_B, 'dividedMonth')

df_test = get_day_type(df_test, 'guess_date')
df_test = addRestdayAfterOneDay(df_test, 'after_restday_one', 0)

df_test_B['after_restday_one'] = df_test_B['date'].map(lambda x: df_test['after_restday_one'][df_test.date == x].values[0])

test_date_set = ['2017-01-03', '2017-02-03', '2017-04-05', '2017-05-02', '2017-05-31', '2017-10-09']
df_test_B = addHolidayAfterOneDayByDate(df_test_B, 'after_holiday_one', test_date_set)

df_test_B = addIsHolidayRestDay(df_test_B, 'is_holi_restday')

test_newYearDay_set = ['2017-01-01', '2017-01-02', '2017-01-03']
df_test_B = addIsNewYearDay(df_test_B, 'is_newYearDay', test_newYearDay_set)

test_ChineseNewYearDay_set = ['2017-01-27', '2017-01-28', '2017-01-29', '2017-01-30', '2017-01-31', '2017-02-01', '2017-02-02']
df_test_B = addIsChineseNewYearDay(df_test_B, 'is_ChineseNewYearDay', test_ChineseNewYearDay_set)

test_NationalDay_set = ['2017-10-01', '2017-10-02', '2017-10-03', '2017-10-04', '2017-10-05', '2017-10-06', '2017-10-07', '2017-10-08']
df_test_B = addIsNationalDay(df_test_B, 'is_NationalDay', test_NationalDay_set)

df_test_B = addPredictBrandsNumber(df_test_B, 'brands_number')

df_test = addIsHoliWeekend(df_test, 'isHolidayWeekend', 0, 0, 0)

df_test_B['isHolidayWeekend'] = df_test_B['date'].map(lambda x: df_test['isHolidayWeekend'][df_test.date == x].values[0])

df_test_B = addIsPureWeekend(df_test_B, 'isPureWeekend')

print(df_test_B.head(10))

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

   date  day_of_week  brand  week guess_date  date_year  date_month  \
0  1250            3      1   197 2016-10-12       2016          10   
1  1250            3      2   197 2016-10-12       2016          10   
2  1250            3      3   197 2016-10-12       2016          10   
3  1250            3      4   197 2016-10-12       2016          10   
4  1250            3      5   197 2016-10-12       2016          10   
5  1250            3      6   197 2016-10-12       2016          10   
6  1250            3      7   197 2016-10-12       2016          10   
7  1250            3      8   197 2016-10-12       2016          10   
8  1250            3      9   197 2016-10-12       2016          10   
9  1250            3     10   197 2016-10-12       2016          10   

   date_property guess_date_str  sale_quantity dividedMonth  \
0              0     2016-10-12          57975        early   
1              0     2016-10-12          57975        early   
2              0     2016-10-

In [42]:
#缩放字段至0-1
def scalerFea(df, cols):
    df.dropna(inplace=True)
    scaler = MinMaxScaler()
    df[cols] = scaler.fit_transform(df[cols].values)
    return df,scaler.scale_

#将相关字段进行归一化处理
def scaleData(train_df, test_df, colSet):
    df = pd.concat([train_df[colSet], test_df[colSet]], axis=0)
    for colName in colSet:
        scaler = MinMaxScaler()
        df[colName] = scaler.fit_transform(df[colName].values)
        train_df[colName + '_scaled'] = df[:len(train_df)][colName]
        test_df[colName + '_scaled'] = df[len(train_df):][colName]
    return train_df, test_df

scaleColSet = ['brands_number', 'week', 'date_year', 'sale_quantity']
# df['week'] = df['date'].map(lambda x: dataDf['week'][dataDf.date == x].values[0])
train_df, test_df = scaleData(full_train_df, df_test_B, scaleColSet)
print(test_df.head(10))

ValueError: Expected 2D array, got 1D array instead:
array=[  3.   3.   3. ...,  10.  10.  10.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [44]:
#添加one-hot编码并保留原字段
def addOneHot(df, colName):
    colDum = pd.get_dummies(df[colName], prefix=colName)
    df = pd.concat([df, colDum], axis=1)
    return df

#将相关字段进行one-hot编码
def oneHotData(train_df, test_df, colSet):
    for colName in colSet:
        train_df = addOneHot(train_df, colName)
        test_df = addOneHot(test_df, colName)
    return train_df, test_df

oneHotColSet = ['day_of_week', 'date_property', 'date_month', 'dividedMonth', 'brand']
train_df, test_df = oneHotData(full_train_df, df_test_B, oneHotColSet)

#因为预测的数据集中3月份为空，因此手动赋值date_month_3
# test_df['date_month_1'] = 0
# test_df['date_month_2'] = 0
# test_df['date_month_3'] = 0
# test_df['date_month_4'] = 0
# test_df['date_month_11'] = 0
# test_df['date_month_12'] = 0

print(test_df.head(10))

   date  day_of_week  brand  week guess_date  date_year  date_month  \
0  1250            3      1   197 2016-10-12       2016          10   
1  1250            3      2   197 2016-10-12       2016          10   
2  1250            3      3   197 2016-10-12       2016          10   
3  1250            3      4   197 2016-10-12       2016          10   
4  1250            3      5   197 2016-10-12       2016          10   
5  1250            3      6   197 2016-10-12       2016          10   
6  1250            3      7   197 2016-10-12       2016          10   
7  1250            3      8   197 2016-10-12       2016          10   
8  1250            3      9   197 2016-10-12       2016          10   
9  1250            3     10   197 2016-10-12       2016          10   

   date_property guess_date_str  sale_quantity    ...    brand_1  brand_2  \
0              0     2016-10-12          57975    ...          1        0   
1              0     2016-10-12          57975    ...          0

In [45]:
#导出训练集
exportResult(train_df, 'prophet_fusai_B_fullfilling_train_feature_set')

#导出测试集A
exportResult(test_df, 'prophet_fusai_B_fullfilling_test_feature_set')